In [9]:


""" class IonImageDatasetWithNormalizationTensors(Dataset):
    def __init__(self, file_path):
        # Load the .h5 file
        self.file = h5py.File(file_path, "r")
        # Extract and store keys
        self.keys = list(self.file.keys())

        # Here, we compute the global mean and std using PyTorch tensors
        self.global_mean, self.global_std = self.compute_global_mean_std()

    def compute_global_mean_std(self):
        # Accumulators for mean and std computation
        all_data = []
        for key in self.keys:
            data = torch.tensor(self.file[key][:], dtype=torch.float32).flatten()
            all_data.append(data)
        all_data = torch.cat(all_data)

        # Compute the global mean and std
        global_mean = torch.mean(all_data)
        global_std = torch.std(all_data)

        return global_mean.item(), global_std.item()

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):
        # Access the dataset using the key
        data = self.file[self.keys[idx]]
        # Convert the data to a torch tensor (for preprocessing)
        image_tensor = torch.tensor(data[:], dtype=torch.float32)

        # Standardization
        image_tensor = (image_tensor - self.global_mean) / self.global_std

        # Placeholder for actual label determination logic
        label = torch.tensor(-1)

        return image_tensor, label

    def close(self):
        self.file.close()


# Initialize the dataset with tensor-based normalization
dataset_normalized_tensors = IonImageDatasetWithNormalizationTensors(file_path)

# Fetch one sample from the dataset to demonstrate normalization with tensors
normalized_tensor_sample_image, normalized_tensor_sample_label = (
    dataset_normalized_tensors[2000]
)

# Showing the shape of one sample image and its mean and std after normalization using tensors
print(
    (
        normalized_tensor_sample_image.shape,
        normalized_tensor_sample_image.mean(),
        normalized_tensor_sample_image.std(),
        normalized_tensor_sample_label,
    )
)

# Remember to close the dataset file when done to avoid resource leaks
dataset_normalized_tensors.close() """

' class IonImageDatasetWithNormalizationTensors(Dataset):\n\n\n\n\n\n\n\n    def __init__(self, file_path):\n\n\n\n\n\n\n\n        # Load the .h5 file\n\n\n\n\n\n\n\n        self.file = h5py.File(file_path, "r")\n\n\n\n\n\n\n\n        # Extract and store keys\n\n\n\n\n\n\n\n        self.keys = list(self.file.keys())\n\n\n\n\n\n\n\n\n        # Here, we compute the global mean and std using PyTorch tensors\n\n\n\n\n\n\n\n        self.global_mean, self.global_std = self.compute_global_mean_std()\n\n\n\n\n\n\n\n\n    def compute_global_mean_std(self):\n\n\n\n\n\n\n\n        # Accumulators for mean and std computation\n\n\n\n\n\n\n\n        all_data = []\n\n\n\n\n\n\n\n        for key in self.keys:\n\n\n\n\n\n\n\n            data = torch.tensor(self.file[key][:], dtype=torch.float32).flatten()\n\n            all_data.append(data)\n\n\n\n\n\n\n\n        all_data = torch.cat(all_data)\n\n\n\n\n\n\n\n\n        # Compute the global mean and std\n\n\n\n\n\n\n\n        global_mean = torch.mean(al

Implement a convolutional neural network to classify images.

In [1]:
%pip install sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

In [10]:
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader


# cropped_ions_file_path = "binary/cropped_ions.h5"

labels_images_file_path = "binary/labels_and_images.pt"  # file path of .pt file
tensor_data = torch.load(labels_images_file_path)
images = tensor_data["images"]
labels = tensor_data["labels"]

# # Assuming you have the input image tensor and label tensor
# images = # (10000, 4, 5, 5) tensor
# labels = # (10000, 4, 1) tensor

# Split the data into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.2, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

# Reshape the input tensor to have the correct shape
X_train = X_train.view(-1, 4, 5, 5)
X_val = X_val.view(-1, 4, 5, 5)
X_test = X_test.view(-1, 4, 5, 5)

# Reshape the label tensor to have the correct shape
y_train = y_train.squeeze(-1).long()
y_val = y_val.squeeze(-1).long()
y_test = y_test.squeeze(-1).long()


# # Define the CNN model
# class CNNModel(nn.Module):
#     def __init__(self):
#         super(CNNModel, self).__init__()
#         self.conv1 = nn.Conv2d(4, 32, kernel_size=2, stride=1, padding=1)
#         self.relu1 = nn.ReLU()
#         self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
#         self.relu2 = nn.ReLU()
#         self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.flatten = nn.Flatten()
#         self.fc1 = nn.Linear(64 * 1 * 1, 128)
#         self.relu3 = nn.ReLU()
#         self.fc2 = nn.Linear(128, 4)
#         self.sigmoid = nn.Sigmoid()

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.relu1(x)
#         x = self.pool1(x)
#         x = self.conv2(x)
#         x = self.relu2(x)
#         x = self.pool2(x)
#         x = self.flatten(x)
#         x = self.fc1(x)
#         x = self.relu3(x)
#         x = self.fc2(x)
#         x = self.sigmoid(x)
#         return x


class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        # Only one convolutional layer without padding and with a kernel size of 2
        self.conv1 = nn.Conv2d(
            4, 32, kernel_size=2, padding=0
        )  # No padding, default stride=1
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

        # Adjust the size in the Linear layer according to the output size from the pooling layer
        # Note: The exact size calculation will depend on the input dimensions to the model
        # Assuming the input dimensions are square and denoted as 'input_size', the calculation would be:
        # output_size = (input_size - 1) / 2  # from conv1 and pool1
        # output_size = output_size / 2  # additional pooling
        # You will need to compute this based on your specific input dimensions and update the number '64' below
        self.fc1 = nn.Linear(32 * 1 * 1, 128)  # Update the 'output_size' accordingly
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 4)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x


# Create the model, optimizer, and loss function
model = CNNModel()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.BCELoss()

# Train the model
num_epochs = 500
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train.float())

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Validation
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs, y_val.float())
    print(
        f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Validation Loss: {val_loss.item():.4f}"
    )

# Evaluate the model on the test set
with torch.no_grad():
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test.float())
    predicted = (test_outputs > 0.5).long()
    test_accuracy = accuracy_score(y_test, predicted)
    print(
        f"Test Loss: {test_loss.item():.4f}, Test Accuracy: {100 * test_accuracy:.2f}%"
    )

RuntimeError: mat1 and mat2 shapes cannot be multiplied (6400x128 and 32x128)